In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import convert_all as crd

DEBUG:h5py._conv:Creating converter from 7 to 5
DEBUG:h5py._conv:Creating converter from 5 to 7
DEBUG:h5py._conv:Creating converter from 7 to 5
DEBUG:h5py._conv:Creating converter from 5 to 7


In [5]:
# get all low_dim data (head&spine velocity control)
raw_dir = "../demonstrations/raw/put_objects_into_bowl_2"
data = crd.raw_to_dict(
    raw_dir,
    ["records.json"],
    video_file_names=None,
    flatten_mode="hdf5",
    concatenater={
        "/observations/qpos": (
            "/observations/pos_f",
            "/observations/endpos_f",

        ),
        "/action": (
            "/observations/pos_t",
            "/observations/endpos_t",
            "/observations/stage",

        ),
    },
    key_filter=[
        "/observations/eef_pose_f",
        "/observations/eef_pose_t",
        "/observations/eff_f",
        "/observations/eff_t",
        "/observations/vel_f",
        "/observations/vel_t",    
    ],

    #eff_f是力反馈，endpos是末端夹爪位置，pos_f是臂的关节数据，eef是末端关节位姿（x,y,z加四元数）vel_t是电流值测定的力反馈，不太准所以不合适加入
    # name_converter={
    #     "//observations/eef_pose_f_left": "/observations/qpos", #注意格式！！！key中有两个//！
    #     "//observations/eef_pose_f_right": "/action",
    # }, #concatenater中已经包含改名的操作，所以这里不需要了

)
print(data["0"]["/observations/qpos"])
print(data["0"]["/action"])
#此时返回的data是一个字典
# data = {
#     "1": {
#         "/observations/qpos": [
#             [0, 1, 2],
#             [3, 4, 5]
#         ]
#     },
#     # 其他键值对...
#}

Data Converting: 100%|██████████| 50/50 [00:00<00:00, 282.88it/s]

[[-0.10166323184967041, -1.0812923908233643, 1.2232013940811157, 1.6962310075759888, -1.3185702562332153, -1.7835888862609863, 0.029158109387794097], [-0.10166323184967041, -1.0816738605499268, 1.2232013940811157, 1.6962310075759888, -1.3227664232254028, -1.7843518257141113, 0.029158109387794097], [-0.10166323184967041, -1.0816738605499268, 1.2232013940811157, 1.6966124773025513, -1.3277256488800049, -1.7877851724624634, 0.029158109387794097], [-0.10166323184967041, -1.0816738605499268, 1.2232013940811157, 1.6969939470291138, -1.3307774066925049, -1.7900739908218384, 0.029158109387794097], [-0.10204470902681351, -1.0816738605499268, 1.2232013940811157, 1.6989012956619263, -1.3368810415267944, -1.793888807296753, 0.029158109387794097], [-0.1028076633810997, -1.0816738605499268, 1.2232013940811157, 1.7057678699493408, -1.3448920249938965, -1.8015183210372925, 0.053681535767270375], [-0.1028076633810997, -1.0816738605499268, 1.2232013940811157, 1.7103456258773804, -1.354047417640686, -1.8

In [6]:
ep_number = len(data)
print(f"Number of episodes: {ep_number}")

Number of episodes: 50


In [10]:
print(list(data["0"].keys()))
print(list(data["1"]["/observations/qpos"][0]))
print(list(data["0"]["/action"][199]))

['/observations/time', '/observations/qpos', '/action']
[-0.00019073777366429567, -0.7661936283111572, 0.7032501697540283, 1.5360113382339478, -0.9653238654136658, -1.5779736042022705, 0.03386462663675283]
[0.04177157208323479, -0.28477150201797485, 0.5899519324302673, 1.8495842218399048, -0.8192187547683716, -1.7591744661331177, 0.00020851036296169905, 8.0]


In [12]:
import os
import cv2
from concurrent.futures import ThreadPoolExecutor

# merge high_dim data and save
raw_dir
video_names = ["0.avi"]
target_dir = f"../data/hdf5/put_objects_into_bowl_2_with_stage/"
low_dim_data = data
name_converter = {"0": "/observations/images/0"}
target_namer = lambda i: f"episode_{i}.hdf5"

compresser = crd.Compresser("jpg", [int(cv2.IMWRITE_JPEG_QUALITY), 50], True)

os.makedirs(target_dir, exist_ok=True)

# get max episode length
episode_lens = []
for low_d in low_dim_data.values():
    episode_lens.append(len(list(low_d.values())[0]))
max_pad_lenth = max(episode_lens)

episode_names = list(low_dim_data.keys())
print(f"Max episode length: {max_pad_lenth}")
print(f"All episodes: {episode_names}")
print(f"episode number: {len(episode_names)}")

Max episode length: 200
All episodes: ['24', '13', '12', '44', '47', '19', '4', '40', '1', '8', '38', '46', '16', '10', '39', '23', '41', '43', '14', '5', '18', '9', '17', '28', '35', '45', '49', '11', '31', '48', '22', '0', '7', '6', '27', '34', '21', '29', '32', '2', '36', '33', '15', '25', '20', '37', '42', '30', '26', '3']
episode number: 50


In [13]:
# save all data
futures = []
with ThreadPoolExecutor(max_workers=25) as executor:

    def save_one(index, ep_name):
        crd.merge_video_and_save(
            low_dim_data[ep_name],
            f"{raw_dir}/{ep_name}",
            video_names,
            crd.save_dict_to_hdf5,
            name_converter,
            compresser,
            f"{target_dir}/" + target_namer(index),
            max_pad_lenth,
        )
        data.pop(ep_name)

    for index, ep_name in enumerate(episode_names):
        futures.append(executor.submit(save_one, index, ep_name))
#执行到这里，所有数据已经保存完毕

DEBUG:root:compressed_len_list: [[12317, 12287, 12329, 12308, 12285, 12387, 12306, 12246, 12258, 12326, 12252, 12287, 12441, 12557, 12629, 12685, 12736, 12778, 12746, 12917, 13073, 13110, 13602, 13765, 13709, 13648, 14048, 14006, 13498, 13376, 12863, 12872, 12654, 12579, 12887, 12640, 12750, 12816, 13149, 13129, 13053, 13028, 12990, 13099, 13188, 13220, 13442, 13689, 13998, 14490, 15077, 15501, 16294, 16664, 16591, 16199, 15840, 15737, 14644, 14129, 13555, 13243, 13067, 12888, 12489, 12323, 12232, 12082, 11864, 11957, 11898, 11903, 11903, 11876, 11859, 11816, 11865, 11832, 11833, 11861, 11873, 11861, 11872, 11960, 12073, 12211, 12325, 12422, 12427, 12531, 12674, 12726, 12783, 12904, 13054, 13092, 13358, 13663, 13866, 13877, 13914, 14084, 14072, 14372, 14160, 14308, 14339, 13848, 13888, 13634, 13497, 13167, 12732, 12498, 12494, 12496, 12463, 12349, 12260, 12370, 12236, 12156, 12176, 12144, 12089, 12068, 12094, 12386, 12808, 13020, 13200, 13321, 13801, 13496, 14097, 14269, 14265, 14308, 

In [ ]:
# fix action
import json
import numpy as np

for i in range(49, 50):
    with open(f"demonstrations/raw/mmk_pick_grape/{i}/low_dim.json", "r") as f:
        data = json.load(f)
        action = data["action"]["spine"]["joint_position"]
        obs = data["observation"]["spine"]["joint_position"]
        # change action
        delta = np.array(obs[1:]) - np.array(obs[:-1])
        # > 0 -> 1, < 0 -> -1
        action = np.sign(delta)
        # 获取不为0的值的index
        index_not0 = np.where(action != 0)
        first_not0 = index_not0[0][0]
        last_not0 = index_not0[0][-1]
        # 从第一个不为0的值开始，到最后一个不为0的值结束，将中间的值设为1
        action[first_not0:last_not0] = 1
        action = action.tolist()
        action.append(action[-1])
        data["action"]["spine"] = {"joint_velocity": action}
    with open(f"demonstrations/raw/mmk_pick_grape/{i}/low_dim.json", "w") as f:
        json.dump(data, f)

In [ ]:
# rename
import json

for i in range(50):
    with open(f"demonstrations/raw/mmk_pick_grape/{i}/low_dim.json", "r") as f:
        data = json.load(f)
        data["action"]["head"] = {"joint_velocity": data["action"]["head"]["joint_velocity"]["joint_position"]}
    with open(f"demonstrations/raw/mmk_pick_grape/{i}/low_dim.json", "w") as f:
        json.dump(data, f)

## Position Control

In [ ]:
# add head and spine to joint_position action
import json

for i in range(58):
    with open(f"demonstrations/raw/mmk_pick_grape/{i}/low_dim.json", "r") as f:
        data = json.load(f)
        obs_spine = data["observation"]["spine"]["joint_position"]
        obs_head = data["observation"]["head"]["joint_position"]
        data["action"]["head"]["joint_position"] = obs_head[1:] + [obs_head[-1]]
        data["action"]["spine"]["joint_position"] = obs_spine[1:] + [obs_spine[-1]]
    with open(f"demonstrations/raw/mmk_pick_grape/{i}/low_dim.json", "w") as f:
        json.dump(data, f)

In [ ]:
# get all low_dim data (head&spine position control)
import convert_all as crd
raw_dir = "demonstrations/raw/mmk_pick_grape"
data = crd.raw_to_dict(
    raw_dir,
    ["low_dim.json"],
    video_file_names=None,
    flatten_mode="hdf5",
    concatenater={
        "/observations/qpos": (
            "/observation/arm/left/joint_position",
            "/observation/eef/left/joint_position",
            "/observation/arm/right/joint_position",
            "/observation/eef/right/joint_position",
            "/observation/head/joint_position",
            "/observation/spine/joint_position"
        ),
        "/action": (
            "/action/arm/left/joint_position",
            "/action/eef/left/joint_position",
            "/action/arm/right/joint_position",
            "/action/eef/right/joint_position",
            "/action/head/joint_position",
            "/action/spine/joint_position"
        ),
    },
    key_filter=[
        "/observation/arm/left/joint_velocity",
        "/observation/arm/right/joint_velocity",
        "/observation/arm/left/joint_torque",
        "/observation/arm/right/joint_torque",
        "/action/head/joint_velocity",
        "/action/spine/joint_velocity"
    ],
)

In [ ]:
ep_number = len(data)
print(f"Number of episodes: {ep_number}")

In [ ]:
# 将spine的observation和action[-120000, 0]缩放到[-2pi, 0]
import numpy as np
for i in range(ep_number):
    qpos_obs = np.array(data[str(i)]["/observations/qpos"])
    qpos_act = np.array(data[str(i)]["/action"])
    qpos_obs[:, -1] = 2 * np.pi * qpos_obs[:, -1] / 120000
    qpos_act[:, -1] = 2 * np.pi * qpos_act[:, -1] / 120000
    data[str(i)]["/observations/qpos"] = qpos_obs.tolist()
    data[str(i)]["/action"] = qpos_act.tolist()